In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import cv2
import json
import os
import matplotlib.pyplot as plt
import random
import seaborn as sns
from keras.models import Sequential
from keras import backend as K
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from PIL import Image

2024-01-19 04:13:25.170585: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 04:13:25.170622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 04:13:25.171778: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 04:13:25.177745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 04:13:25.767391: W tensorflow/compiler/tf2

In [2]:
import zipfile
zip = zipfile.ZipFile('archive.zip')
zip.extractall('face_detection/')
zip.close()


FileNotFoundError: [Errno 2] No such file or directory: 'archive.zip'

In [18]:
with_mask_files = os.listdir('face_detection/data/with_mask/')
print(with_mask_files[0:5])

['with_mask_1708.jpg', 'with_mask_3551.jpg', 'with_mask_1146.jpg', 'with_mask_3320.jpg', 'with_mask_2797.jpg']


In [3]:
without_mask_files = os.listdir('face_detection/data/without_mask/')
print(without_mask_files[0:5])

['without_mask_1469.jpg', 'without_mask_2817.jpg', 'without_mask_2304.jpg', 'without_mask_2374.jpg', 'without_mask_2871.jpg']


In [4]:
#create labels

with_mask_labels = [1]*3725

without_mask_labels = [0] * 3828

In [5]:
labels = with_mask_labels + without_mask_labels

In [6]:
#display with mask image

img = cv2.imread('face_detection/data/with_mask/with_mask_100.jpg')
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows


In [ ]:
img2 = cv2.imread('face_detection/data/without_mask/without_mask_100.jpg')
cv2.imshow('image2', img2)
cv2.waitKey(0)
cv2.destroyAllWindows

<function destroyAllWindows>

In [19]:
#Image Processing
#Resize the image
#Convert the images to numpy arrays
with_mask_path = 'face_detection/data/with_mask/'
data = []
for img_file in with_mask_files:
    image = Image.open(with_mask_path + img_file)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
    
without_mask_path = 'face_detection/data/without_mask/'
for img_file in without_mask_files:
    image = Image.open(without_mask_path + img_file)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)

/home/rutpatel/Documents/deeplearning/dl/lib/python3.10/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [22]:
#converting image list and label list to numpy arrays
X = np.array(data)
Y = np.array(labels)

In [8]:
type(X)

numpy.ndarray

In [23]:
type(Y)

numpy.ndarray

In [24]:
#train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [25]:
print(X.shape, X_train.shape, X_test.shape)

(7553, 128, 128, 3) (6042, 128, 128, 3) (1511, 128, 128, 3)


In [26]:
#scaling the data
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [27]:
#Building CNN 
from keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet',
                include_top=False,
                input_shape=(128, 128, 3))

conv_base.trainable=False

In [28]:
model = keras.Sequential()

model.add(conv_base)

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_3 (Dense)             (None, 128)               1048704   
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 15771713 (60.16 MB)
Trainable params: 1057025 (4.03 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
history = model.fit(X_train_scaled, Y_train, epochs=5, validation_split=0.1)

2024-01-19 04:16:20.890844: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1068957696 exceeds 10% of free system memory.
2024-01-19 04:16:21.843092: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1068957696 exceeds 10% of free system memory.


Epoch 1/5


2024-01-19 04:16:22.950659: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-19 04:16:23.103275: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-19 04:16:24.022254: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-19 04:16:24.690456: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc18d6bc360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-19 04:16:24.690492: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-01-19 04:16:24.701419: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1705617984.784482   19333 devic

170/170 [==============================] - ETA: 0s - loss: 0.1670 - accuracy: 0.9318

2024-01-19 04:16:31.955697: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 118947840 exceeds 10% of free system memory.
2024-01-19 04:16:32.067276: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 118947840 exceeds 10% of free system memory.


170/170 [==============================] - 11s 46ms/step - loss: 0.1670 - accuracy: 0.9318 - val_loss: 0.0745 - val_accuracy: 0.9769
Epoch 2/5
170/170 [==============================] - 7s 40ms/step - loss: 0.0582 - accuracy: 0.9785 - val_loss: 0.0705 - val_accuracy: 0.9736
Epoch 3/5
170/170 [==============================] - 7s 40ms/step - loss: 0.0347 - accuracy: 0.9871 - val_loss: 0.1237 - val_accuracy: 0.9570
Epoch 4/5
170/170 [==============================] - 7s 41ms/step - loss: 0.0184 - accuracy: 0.9956 - val_loss: 0.1729 - val_accuracy: 0.9455
Epoch 5/5
170/170 [==============================] - 7s 41ms/step - loss: 0.0153 - accuracy: 0.9952 - val_loss: 0.2475 - val_accuracy: 0.9306
